## Drosophila Examples for Pairwise Alignment

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
sys.path.insert(0,'../')
import morpho
import matplotlib.pyplot as plt
import scanpy
import anndata
import numpy as np
from jupyterthemes import jtplot
jtplot.style(theme="onedork")
import spateo as st
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
slice_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
%load_ext autoreload
%autoreload 2

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [4]:
# cellbin data
cellbin_slices = st.sample_data.drosophila()

# binning data
# binning_slices = st.sample_data.drosophila(filename="E7-9h_bin20_h5ad.zip")[3:7]

|-----> Downloading data to ./data/E7-9h_cellbin_tdr_v2.h5ad
https://www.dropbox.com/s/bvstb3en5kc6wui/E7-9h_cellbin_tdr_v2.h5ad?dl=1


KeyboardInterrupt: 

In [ ]:
data_dir='../data/cellbin_h5ad'
slices = [anndata.read_h5ad(os.path.join(data_dir, "E8-10_b_S{:0>2d}.h5ad".format(index))) for index in range(6,30)]

In [ ]:
cellbin_slices_sample = [slice.copy() for slice in slices]
align_cellbin_slices = st.tl.models_align(
    models=cellbin_slices_sample,
    spatial_key="spatial",
    key_added="align_spatial",
    numItermax=200,
    numItermaxEmd=500000,
    device="2"
)

In [ ]:
%matplotlib inline
st.pl.multi_slices(
    slices=align_cellbin_slices.copy(),
    slices_key="slices",
    spatial_key="align_spatial",
    point_size=50,
    ax_height=0.6,
    ncols=2,
)

## 3D model

In [ ]:
data_dir='../data/drosophila_data'
slice_A = scanpy.read_h5ad(os.path.join(data_dir,'E7-9h_cellbin_tdr_v2.h5ad'))
# slice_B = scanpy.read_h5ad(os.path.join(data_dir,'E9-10h_cellbin_tdr_v1.h5ad'))

In [2]:
data_dir='../data/cellbin_3D'
slice_A = scanpy.read_h5ad(os.path.join(data_dir,'E7-8h_cellbin_v1.h5ad'))
# slice_B = scanpy.read_h5ad(os.path.join(data_dir,'E8-9h_cellbin_v1.h5ad'))

In [3]:
slice_A

AnnData object with n_obs × n_vars = 15649 × 6868
    obs: 'area', 'slices', 'Annotation_1_cell_type', 'Annotation_2_tissue', 'Annotation_3_germ_layer', 'actual_time'
    uns: '__type'
    obsm: 'X_pca', 'X_umap', 'align_spatial', 'bbox', 'contour', 'spatial', 'tdr_spatial'
    layers: 'counts_X'

In [ ]:
slice_A.obs['Annotation_2_tissue']

In [4]:
tissue_colormap = {
    "CNS": "#5A2686",
    "midgut": "#DC143C",
    "amnioserosa": "#FFDD00",
    "salivary gland": "#FFA500",

    "epidermis": "#ADD8E6",
    "muscle": "#1AAB27",
    "trachea": "#7F90F0",
    "hemolymph": "#CD5C5C",
    "yolk": "#8B0000",

    "mesoderm": "#CD5C5C",
    "unknown":"#7F90F0",
}
embryo_pc = st.tdr.construct_pc(
    adata=slice_A.copy(),
    spatial_key="align_spatial",
    groupby="Annotation_2_tissue",
    key_added="tissue",
    colormap=tissue_colormap,
)
embryo_pc.point_data

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : tissue_rgba
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    tissue_rgba             float32  (15649, 4)           SCALARS
    tissue                  <U11     (15649,)
    obs_index               <U18     (15649,)

In [5]:
%matplotlib inline
st.pl.three_d_plot(
    model=embryo_pc,
    key="tissue",
    model_style="points",
    model_size=3,
    jupyter="static",
    window_size=(512, 512)
)

: 

: 

### Exact Alignment on CPU

In [ ]:
i = 2
BAS_slices = [slice.copy() for slice in slices]
sliceA = BAS_slices[i]
sliceB = BAS_slices[i + 1]
align_model_A, align_model_B, P = morpho.morpho_align_pairwise(
            model_ref=sliceA,
            model=sliceB,
            spatial_key="spatial",
            key_added="align_spatial",
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=False,
            mode="SN",
            device="2",
            max_iter=100,
            dtype='float64',
            beta2=0.1,
            verbose=True,
            iter_key_added='iter_spatial',
            keep_size=True,
        )

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 3,figsize=(30,5))
plot_slices(align_model_A, align_model_B,ax,'align_spatial',slice_colors, 10)
plt.tight_layout()
plt.show()

In [ ]:
BAS_slices = [slice.copy() for slice in slices]
align_models = morpho.morpho_align(
            models=BAS_slices,
            spatial_key="spatial",
            key_added="align_spatial",
            iter_key_added='iter_spatial',
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=False,
            mode="SN",
            device="2",
            max_iter=100,
            dtype='float64',
            beta2=0.1,
            verbose=True,
            )

In [ ]:
morpho.two_d_pairwise_animate(
    model_ref=align_model_A,
    model=align_model_B,
    fps=20
)

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(1, 3,figsize=(30,5))
morpho.plot_pairewise_slices(
    sliceA_align=align_model_A,
    sliceB_align=align_model_B,
    axs = axs,
    spatial_key='spatial',
    )


In [ ]:
%matplotlib auto
i = 9
BAS_slices = [slice.copy() for slice in slices]
sliceA = BAS_slices[i]
sliceB = BAS_slices[i + 1]
align_models = morpho.morpho_align(
            models=BAS_slices,
            spatial_key="spatial",
            key_added="align_spatial",
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=False,
            mode="SN",
            device="2",
            max_iter=100,
            dtype='float64',
            beta2=0.1,
            verbose=True,
            )

In [ ]:
BAS_slices = [slice.copy() for slice in slices]
align_models, coarse_align_models = morpho.morpho_align_global(
            models=BAS_slices,
            spatial_key="spatial",
            key_added="align_spatial",
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=False,
            mode="SN",
            device="2",
            max_iter=100,
            dtype='float64',
            beta2=0.1,
            verbose=True,
            )

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(len(coarse_align_models)-1, 3,figsize=(15,(len(coarse_align_models)-1)*3))
for i in range(len(coarse_align_models)-1):
    sliceA = coarse_align_models[i]
    sliceB = coarse_align_models[i + 1]
    plot_slices(sliceA, sliceB,axs[i],'align_spatial',slice_colors,5)
plt.tight_layout()
plt.show()


In [ ]:
%matplotlib inline
fig, axs = plt.subplots(len(align_models)-1, 3,figsize=(15,(len(align_models)-1)*3))
for i in range(len(align_models)-1):
    sliceA = align_models[i]
    sliceB = align_models[i + 1]
    plot_slices(sliceA, sliceB,axs[i],'align_spatial',slice_colors,20)


In [ ]:
%matplotlib inline
fig, axs = plt.subplots(len(coarse_align_models)-1, 3,figsize=(15,(len(coarse_align_models)-1)*3))
for i in range(len(coarse_align_models)-1):
    sliceA = coarse_align_models[i]
    sliceB = coarse_align_models[i + 1]
    plot_slices(sliceA, sliceB,axs[i],'align_spatial',slice_colors,20)


In [ ]:
%matplotlib inline
morpho.plot_multi_slices(
    align_models,
    spatial_key='align_spatial'
    )


In [ ]:
align_models_PASTE = morpho.models_align(
        models=BAS_slices,
        spatial_key="spatial",
        key_added="align_spatial",
        device='0',
    )

In [ ]:
%matplotlib inline
morpho.plot_multi_slices(
    align_models_PASTE,
    spatial_key='align_spatial'
    )


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1, 3,figsize=(30,5))
plot_slices(align_models[0], align_models[1],ax,'align_spatial',slice_colors, 10)
plt.tight_layout()
plt.show()

In [ ]:
BAS_slices = [slice.copy() for slice in slices]
%matplotlib inline
fig, axs = plt.subplots(len(slices)-1, 3,figsize=(15,(len(slices)-1)*3))
for i in range(len(slices)-1):
    sliceA = BAS_slices[i]
    sliceB = BAS_slices[i + 1]
    sliceA_align, sliceB_align, P = morpho.morpho_align_pairwise(
                model=sliceA,
                model_ref=sliceB,
                spatial_key="spatial",
                key_added="align_spatial",
                select_high_exp_genes=False,
                normalize_c=True,
                normalize_g=False,
                mode="SN",
                max_iter=100,
                vis_optimiation=False,
                print_info=False,
                use_gpu = True,
                use_SVI = False,
                batch_size_param = 100,
                beta = 0.05,
                beta2 = 0.1,
                lambdaVF = 1e2,
                )
    plot_slices(sliceA_align, sliceB_align,axs[i],'align_spatial',slice_colors,20)


### Alignment with SVI on CPU

In [ ]:
%matplotlib auto
sliceA_align_SVI, sliceB_align_SVI, P = morpho.morpho_align_pairwise(
            model=sliceA,
            model_ref=sliceB,
            spatial_key="spatial",
            key_added="align_spatial",
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=True,
            mode="SN",
            max_iter=100,
            vis_optimiation=False,
            print_info=True,
            use_gpu = False,
            use_SVI = True,
            batch_size_param = 100,
            beta = 0.05,
            )


### Exact Alignment on GPU

In [ ]:
%matplotlib auto
sliceA_align_GPU, sliceB_align_GPU, P = morpho.morpho_align_pairwise(
            model=sliceA,
            model_ref=sliceB,
            spatial_key="spatial",
            key_added="align_spatial",
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=True,
            mode="SN",
            max_iter=100,
            vis_optimiation=True,
            print_info=True,
            use_gpu = True,
            use_SVI = False,
            batch_size_param = 100,
            beta = 0.05,
            )


### Alignment with SVI on GPU

In [ ]:
%matplotlib auto
sliceA_align_GPU_SVI, sliceB_align_GPU_SVI, P = morpho.morpho_align_pairwise(
            model=sliceA,
            model_ref=sliceB,
            spatial_key="spatial",
            key_added="align_spatial",
            select_high_exp_genes=False,
            normalize_c=True,
            normalize_g=True,
            mode="SN",
            max_iter=100,
            vis_optimiation=False,
            print_info=True,
            use_gpu = True,
            use_SVI = True,
            batch_size_param = 100,
            beta = 0.05,
            )


### Paste on GPU

In [ ]:
import paste as pst
import ot
import time
sliceA_paste, sliceB_paste = slices[i].copy(), slices[i+1].copy()
t1 = time.time()
pi = pst.pairwise_align(sliceA_paste, sliceB_paste, use_gpu=True,backend = ot.backend.TorchBackend())
# pi = pst.pairwise_align(sliceA_paste, sliceB_paste, use_gpu=False)
pis = [pi]
new_slices = pst.stack_slices_pairwise([sliceA_paste, sliceB_paste], pis)
t2 = time.time()
print("Paste costs {:.5} s".format(t2-t1))